In [1]:
import scipy.io
import numpy as np
import pandas as pd
import scipy.stats

**Task** **1**

In [3]:
data = scipy.io.loadmat('train_data.mat')

In [4]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'data', 'label'])

In [7]:
data["label"]

array([[3, 3, 3, ..., 7, 7, 7]])

In [6]:
data_features = []
feature_1_list = []
feature_2_list = []

In [7]:
for i in range(len(data['data'])):
    feature_1 = np.mean(data['data'][i].flatten()).tolist()
    feature_2 = np.std(data['data'][i].flatten())
    # data_features.append([feature_1, feature_2])
    feature_1_list.append(feature_1)
    feature_2_list.append(feature_2)

In [8]:
M1 = np.mean(feature_1_list)
S1 = np.std(feature_1_list)

M2 = np.mean(feature_2_list)
S2 = np.std(feature_2_list)

In [9]:
M1,M2

(32.50435107148159, 76.44042397326842)

In [10]:
S1,S2

(9.349449452973936, 10.509725393032216)

In [11]:
Y1, Y2 = np.zeros(len(data['data'])), np.zeros(len(data['data']))
for i in range(len(data['data'])):
    Y1[i]=(feature_1_list[i]-M1)/S1
    Y2[i]=(feature_2_list[i]-M2)/S2

In [12]:
Y1, M1,S1

(array([ 0.15069489, -0.97386658, -0.60346935, ..., -0.80933654,
        -0.42215888,  0.14114505]), 32.50435107148159, 9.349449452973936)

In [13]:
Y2, M2,S2

(array([ 0.12996069, -0.93473171, -0.6292426 , ..., -0.83222958,
        -0.31853427,  0.41329776]), 76.44042397326842, 10.509725393032216)

## **Task** **2**

In [14]:
#image 3
class0=[]
#image 7
class1=[]

In [15]:
for i in range(len(data['data'])):
    if data['label'][0][i] == 3:
        class0.append(i)
    elif data['label'][0][i]==7:
        class1.append(i)

In [16]:
# For Image 3
splitY1_class_0 = [Y1[i] for i in class0]
splitY2_class_0 = [Y2[i] for i in class0]
# For Image 7
splitY1_class_1 = [Y1[i] for i in class1]
splitY2_class_1 = [Y2[i] for i in class1]

In [17]:
meanOfY1_class_0 , meanOfY2_class_0 = np.mean(splitY1_class_0), np.mean(splitY2_class_0)
meanOfY1_class_1 , meanOfY2_class_1 = np.mean(splitY1_class_1), np.mean(splitY2_class_1)

In [18]:
print(meanOfY1_class_0 , meanOfY2_class_0)
print(meanOfY1_class_1 , meanOfY2_class_1)

0.3768799644438678 0.31851854940980523
-0.3690000405943126 -0.31185886422934433


In [19]:
std_Y1_class_0, std_Y2_class_0 = np.std(splitY1_class_0), np.std(splitY2_class_0)
print(std_Y1_class_0, std_Y2_class_0)

std_Y1_class_1, std_Y2_class_1 = np.std(splitY1_class_1), np.std(splitY2_class_1)
print(std_Y1_class_1, std_Y2_class_1)

1.0242585609232964 0.9799897057943019
0.8226125232643172 0.9177161877589698


In [20]:
mu_class_0 = np.array([meanOfY1_class_0 , meanOfY2_class_0])
mu_class_1 = np.array([meanOfY1_class_1 , meanOfY2_class_1])
mu_class_1

array([-0.36900004, -0.31185886])

In [21]:
print(mu_class_0)
print(mu_class_1)

[0.37687996 0.31851855]
[-0.36900004 -0.31185886]


In [22]:
# Discriminant variables here is the covariance matrix
discriminant_class_0 = 0
discriminant_class_1 = 0

In [23]:
for i in class0:
    y_class_0 = np.array([Y1[i], Y2[i]])
    term_class_1 = (y_class_0-mu_class_0)
    discriminant_class_0 += term_class_1 * term_class_1[None].T
discriminant_class_0 = discriminant_class_0/len(class0)
discriminant_class_0

array([[1.0491056 , 0.98717364],
       [0.98717364, 0.96037982]])

In [24]:
for i in class1:
    y_class_1 = np.array([Y1[i], Y2[i]])
    term_class_1 = (y_class_1-mu_class_1)
    discriminant_class_1 += term_class_1 * term_class_1[None].T
discriminant_class_1 = discriminant_class_1/len(class1)
discriminant_class_1

array([[0.67669136, 0.74435619],
       [0.74435619, 0.842203  ]])

# Estimating Density

In [25]:
mu_class_0,discriminant_class_0

(array([0.37687996, 0.31851855]), array([[1.0491056 , 0.98717364],
        [0.98717364, 0.96037982]]))

In [26]:
mu_class_1,discriminant_class_1

(array([-0.36900004, -0.31185886]), array([[0.67669136, 0.74435619],
        [0.74435619, 0.842203  ]]))

## Task 3

In [27]:
##P(E) for P(3)=P(7)=0.5

In [28]:
prob_w1 = np.zeros(len(data['data']))
prob_w2 = np.zeros(len(data['data']))

In [29]:
from scipy.stats import multivariate_normal
cond_image_3 = multivariate_normal.pdf(np.array([Y1, Y2]).T, mu_class_0, discriminant_class_0)
cond_image_7 = multivariate_normal.pdf(np.array([Y1, Y2]).T, mu_class_1, discriminant_class_1)
tot_prob_x=0.5*cond_image_3+0.5*cond_image_7
prob_w1_case_1 = (0.5 * cond_image_3)/tot_prob_x
prob_w2_case_1 = (0.5 * cond_image_7)/tot_prob_x

In [30]:
prob_w1_case_1,prob_w2_case_1

(array([0.53955864, 0.28232707, 0.32754567, ..., 0.29195989, 0.30544148,
        0.22810482]),
 array([0.46044136, 0.71767293, 0.67245433, ..., 0.70804011, 0.69455852,
        0.77189518]))

In [31]:
np.array([Y1, Y2]).T

array([[ 0.15069489,  0.12996069],
       [-0.97386658, -0.93473171],
       [-0.60346935, -0.6292426 ],
       ...,
       [-0.80933654, -0.83222958],
       [-0.42215888, -0.31853427],
       [ 0.14114505,  0.41329776]])

In [32]:
P_Error_Train_case_1 = 0
for i in range(len(data['data'])):
    if prob_w1_case_1[i] > prob_w2_case_1[i]:
        P_Error_Train_case_1 += prob_w2_case_1[i]
    else:
        P_Error_Train_case_1 += prob_w1_case_1[i]

In [33]:
print(f"Error for P(3)=P(7)=0.5 is {P_Error_Train_case_1/len(data['data'])}")

Error for P(3)=P(7)=0.5 is 0.2632324599044005


In [34]:
P_Error_Train_case_1

3039.808446976017

In [35]:
#P(E) for P(3)= 0.3 and P(7)=0.7

In [36]:
prob_w1_case_2 = (0.3 * cond_image_3)/tot_prob_x
prob_w2_case_2 = (0.7 * cond_image_7)/tot_prob_x

In [37]:
prob_w1_case_2,prob_w2_case_2

(array([0.32373518, 0.16939624, 0.1965274 , ..., 0.17517594, 0.18326489,
        0.13686289]),
 array([0.64461791, 1.0047421 , 0.94143606, ..., 0.99125615, 0.97238193,
        1.08065325]))

In [38]:
P_Error_Train_case_2 = 0
for i in range(len(data['data'])):
    if prob_w1_case_2[i] > prob_w2_case_2[i]:
        P_Error_Train_case_2 += prob_w2_case_2[i]
    else:
        P_Error_Train_case_2 += prob_w1_case_2[i]

In [39]:
print(f"Error for P(3)=0.3 and P(7)=0.7 is {P_Error_Train_case_2/len(data['data'])}")

Error for P(3)=0.3 and P(7)=0.7 is 0.1808480157167703


In [40]:
P_Error_Train_case_2

2088.4328854972637

In [41]:
# Performing for Test Data

In [42]:
test_data = data = scipy.io.loadmat('test_data.mat')

In [43]:
test_data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'data', 'label'])

In [44]:
len(test_data['data'])

2886

In [45]:
test_data['label']

array([[3, 3, 3, ..., 7, 7, 7]])

In [46]:
data_features_test = []
feature_test_1_list = []
feature_test_2_list = []

In [47]:
for i in range(len(test_data['data'])):
    feature_test_1 = np.mean(test_data['data'][i].flatten()).tolist()
    feature_test_2 = np.std(test_data['data'][i].flatten())
    data_features_test.append([feature_1, feature_2])
    feature_test_1_list.append(feature_test_1)
    feature_test_2_list.append(feature_test_2)

In [48]:
M1_test = np.mean(feature_test_1_list)
S1_test = np.std(feature_test_1_list)

M2_test = np.mean(feature_test_2_list)
S2_test = np.std(feature_test_2_list)

In [49]:
M1_test,M2_test

(33.23068083782369, 77.1949417103036)

In [50]:
S1_test,S2_test

(9.807076720815555, 10.874661388710518)

In [63]:
Y1_test, Y2_test = np.zeros(len(test_data['data'])), np.zeros(len(test_data['data']))
for i in range(len(test_data['data'])):
    Y1_test[i]=(feature_test_1_list[i]-M1)/S1
    Y2_test[i]=(feature_test_2_list[i]-M2)/S2

In [139]:
Y1_test, Y2_test

(array([ 1.41659394,  0.41809031,  1.40213276, ..., -1.03157488,
        -0.09323522, -1.35831572]),
 array([ 1.29177314,  0.5888807 ,  1.2132882 , ..., -1.07742787,
         0.12317551, -1.53804101]))

In [65]:
##P(E) for P(3)=P(7)=0.5

In [145]:
prob_test_w1_case1 = np.zeros(len(test_data['data']))
prob_test_w2_case1 = np.zeros(len(test_data['data']))

In [146]:
cond_test_image3 = multivariate_normal.pdf(np.array([Y1_test, Y2_test]).T, mu_class_0, discriminant_class_0)
cond_test_image7 = multivariate_normal.pdf(np.array([Y1_test, Y2_test]).T, mu_class_1, discriminant_class_1)


In [147]:
cond_test_image7

array([0.00747294, 0.77937474, 0.00272436, ..., 0.88812906, 0.82533697,
       0.40863947])

In [148]:
tot_prob_test=(0.5*cond_test_image_3)+(0.5*cond_test_image_7)
tot_prob_test

array([0.26520571, 0.57635776, 0.2468451 , ..., 0.60122059, 0.56219829,
       0.2513826 ])

In [149]:
prob_test_w1_case1 = (0.5 * cond_test_image_3)/tot_prob_test_x
print(probTest_w1_case_1)
prob_test_w2_case1 = (0.5 * cond_test_image_7)/tot_prob_test_x
print(probTest_w2_case_1)

[0.98591105 0.32387937 0.99448164 ... 0.26139501 0.26597342 0.18721609]
[0.01408895 0.67612063 0.00551836 ... 0.73860499 0.73402658 0.81278391]


In [150]:
P_Error_Test_case_1 = 0
for i in range(len(test_data['data'])):
    if prob_test_w1_case1[i] > prob_test_w2_case1[i]:
        P_Error_Test_case_1 += prob_test_w2_case1[i]
    else:
        P_Error_Test_case_1 += prob_test_w1_case1[i]

In [151]:
len(test_data['data'])

2886

In [152]:

print(f"P(Error) for P(3)=P(7)=0.5 is {P_Error_Test_case_1/len(test_data['data'])}")

P(Error) for P(3)=P(7)=0.5 is 0.2550935170023689


In [153]:
P_Error_Test_case_1

736.1998900688366

In [154]:
##P(E) for P(3)=0.3 and P(7)=0.7

In [160]:
prob_test_w1_case2 = np.zeros(len(test_data['data']))
prob_test_w2_case2 = np.zeros(len(test_data['data']))

In [161]:
prob_test_w1_case2 = (0.3 * cond_test_image_3)/tot_prob_test_x
prob_test_w2_case2 = (0.7 * cond_test_image_7)/tot_prob_test_x

In [162]:
P_Error_Test_case_2 = 0
for i in range(len(test_data['data'])):
    if prob_test_w1_case2[i] > prob_test_w2_case2[i]:
        P_Error_Test_case_2 += prob_test_w2_case2[i]
    else:
        P_Error_Test_case_2 += prob_test_w1_case2[i]

In [163]:
print(f"P(Error) for P(3)=0.3 and P(7)=0.7 is {P_Error_Test_case_2/len(test_data['data'])}")

P(Error) for P(3)=0.3 and P(7)=0.7 is 0.17567475918338987


In [164]:
P_Error_Test_case_2

506.9973550032632